Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

In [6]:
batch_size = 128
nodes = 1024
reg_coeff = 6*1e-5

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  w1 = weight_variable([image_size * image_size, nodes])
  b1 = bias_variable([nodes])
  w2 = weight_variable([nodes, num_labels])
  b2 = bias_variable([num_labels])
    
  # Hidden Layer
  h1 = tf.nn.relu(tf.matmul(tf_train_dataset, w1) + b1)

  # L2 regularization for the fully connected parameters.
  regularizers = (tf.nn.l2_loss(w2) +
              tf.nn.l2_loss(w1))
  
  # Training computation.
  logits = tf.matmul(h1, w2) + b2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Add the regularization term to the loss.
  loss += reg_coeff * regularizers
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, w1) + b1), w2) + b2)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, w1) + b1), w2) + b2)

In [7]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size),:]
    batch_labels = train_labels[offset:(offset + batch_size),:]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.663588
Minibatch accuracy: 5.5%
Validation accuracy: 36.2%
Minibatch loss at step 500: 0.591637
Minibatch accuracy: 89.1%
Validation accuracy: 85.1%
Minibatch loss at step 1000: 0.762194
Minibatch accuracy: 82.0%
Validation accuracy: 86.2%
Minibatch loss at step 1500: 0.448162
Minibatch accuracy: 93.0%
Validation accuracy: 86.0%
Minibatch loss at step 2000: 0.449848
Minibatch accuracy: 93.8%
Validation accuracy: 87.6%
Minibatch loss at step 2500: 0.518425
Minibatch accuracy: 90.6%
Validation accuracy: 87.8%
Minibatch loss at step 3000: 0.550283
Minibatch accuracy: 87.5%
Validation accuracy: 88.2%
Test accuracy: 94.0%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [8]:
batch_size = 16

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Restrict offset to only three values
    offset = (step%100)*batch_size
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size),:]
    batch_labels = train_labels[offset:(offset + batch_size),:]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.450202
Minibatch accuracy: 9.4%
Validation accuracy: 31.9%
Minibatch loss at step 500: 0.464692
Minibatch accuracy: 93.0%
Validation accuracy: 83.2%
Minibatch loss at step 1000: 0.278383
Minibatch accuracy: 97.7%
Validation accuracy: 84.1%
Minibatch loss at step 1500: 0.223143
Minibatch accuracy: 99.2%
Validation accuracy: 84.7%
Minibatch loss at step 2000: 0.202019
Minibatch accuracy: 100.0%
Validation accuracy: 84.7%
Minibatch loss at step 2500: 0.192235
Minibatch accuracy: 100.0%
Validation accuracy: 84.7%
Minibatch loss at step 3000: 0.186365
Minibatch accuracy: 99.2%
Validation accuracy: 84.8%
Test accuracy: 91.5%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [9]:
batch_size = 128
nodes = 1024
reg_coeff = 6*1e-5

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(None, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(None, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  w1 = weight_variable([image_size * image_size, nodes])
  b1 = bias_variable([nodes])
  w2 = weight_variable([nodes, num_labels])
  b2 = bias_variable([num_labels])
  
  # Hidden Layer
  h1 = tf.nn.relu(tf.matmul(tf_train_dataset, w1) + b1)

  # Dropout
  #keep_prob = tf.placeholder(tf.float32)
  h_drop = tf.nn.dropout(h1, 0.5)
  
  # Training computation.
  logits = tf.matmul(h_drop, w2) + b2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, w1) + b1), w2) + b2)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, w1) + b1), w2) + b2)

In [10]:
batch_size = 128

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Restrict offset to only three values
    offset = (step%100)*batch_size
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size),:]
    batch_labels = train_labels[offset:(offset + batch_size),:]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 5.696831
Minibatch accuracy: 3.9%
Validation accuracy: 35.6%
Minibatch loss at step 500: 0.605023
Minibatch accuracy: 82.0%
Validation accuracy: 84.0%
Minibatch loss at step 1000: 0.387361
Minibatch accuracy: 85.9%
Validation accuracy: 84.6%
Minibatch loss at step 1500: 0.437044
Minibatch accuracy: 86.7%
Validation accuracy: 84.8%
Minibatch loss at step 2000: 0.307170
Minibatch accuracy: 90.6%
Validation accuracy: 84.9%
Minibatch loss at step 2500: 0.216534
Minibatch accuracy: 94.5%
Validation accuracy: 85.2%
Minibatch loss at step 3000: 0.114713
Minibatch accuracy: 96.9%
Validation accuracy: 84.6%
Test accuracy: 91.3%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [1]:
batch_size = 128
nodes1 = 1024
nodes2 = 1024
reg_coeff = 8*1e-5

graph = tf.Graph()
with graph.as_default():
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(None, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(None, num_labels))
  
  # Variables.
  w1 = weight_variable([image_size * image_size, nodes1])
  b1 = bias_variable([nodes1])
  w2 = weight_variable([nodes1, nodes2])
  b2 = bias_variable([nodes2])
  w3 = weight_variable([nodes2, num_labels])
  b3 = bias_variable([num_labels])
  prob = tf.placeholder(tf.float32)
    
  # Hidden Layer1
  h1 = tf.nn.relu(tf.matmul(tf_train_dataset, w1) + b1)
  # Dropout1
  h_d1 = tf.nn.dropout(h1, prob)

  # Hidden Layer2
  h2 = tf.nn.relu(tf.matmul(h_d1, w2) + b2)
  # Dropout2
  h_d2 = tf.nn.dropout(h2, prob)   
    
  # L2 regularization for the fully connected parameters.
  regularizers = (tf.nn.l2_loss(w2) + tf.nn.l2_loss(w1) +
    tf.nn.l2_loss(w3))
  
  # Training computation.
  logits = tf.matmul(h_d2, w3) + b3
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Add the regularization term to the loss.
  loss += reg_coeff * regularizers
    
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 100000, 0.96, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  correct_prediction = tf.equal(tf.argmax(train_prediction,1), tf.argmax(tf_train_labels,1))
  accurate = tf.reduce_mean(tf.cast(correct_prediction, "float"))*100

NameError: name 'tf' is not defined

In [ ]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    #print(step)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size),:]
    batch_labels = train_labels[offset:(offset + batch_size),:]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, prob: 0.5}
    _, l, _ = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accurate.eval(feed_dict = {
                    tf_train_dataset : batch_data, tf_train_labels : batch_labels, prob: 1}))
      print("Validation accuracy: %.1f%%" % accurate.eval(feed_dict = {
                tf_train_dataset : valid_dataset, tf_train_labels : valid_labels, prob: 1}))
  print("Test accuracy: %.1f%%" % accurate.eval(feed_dict = {
            tf_train_dataset : test_dataset, tf_train_labels : test_labels, prob: 1}))

Initialized
Minibatch loss at step 0: 15.378896
Minibatch accuracy: 23.4%
Validation accuracy: 14.4%